# Declare necessary libraries

In [121]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss

# Loading and standardization data

In [122]:

import pandas as pd

def load_prepare_data(file_path):
    # Đọc dòng đầu tiên để xác định dấu phân tách
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline()
    
    # Xác định dấu phân tách (';' hoặc ',')
    sep = ';' if ';' in first_line else ','
    
    # Đọc file CSV với dấu phân tách đã xác định
    try:
        df = pd.read_csv(file_path, sep=sep)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None
    
    print(f"Processing file: {file_path}")
    print("Columns:", df.columns.tolist())
    
    # Định nghĩa các cột cần giữ cho từng loại file
    columns_to_keep = []
    
    # Trường hợp 1: Dữ liệu có cột 'timeOpen', 'timeClose', 'name', 'close', ...
    if {'timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap', 'timestamp'}.issubset(df.columns):
        columns_to_keep = ['timeClose', 'close']
    
    # Trường hợp 2: Dữ liệu có cột 'Date', 'Open', 'High', 'Low', 'Close', ...
    elif {'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'}.issubset(df.columns):
        columns_to_keep = ['Date', 'Close']
    
    # Trường hợp 3: Dữ liệu có cột 'Date', 'Price', 'Open', ...
    elif {'Date', 'Price', 'Open', 'High', 'Low', 'Change %'}.issubset(df.columns):
        columns_to_keep = ['Date', 'Price']
    
    # Lọc các cột cần thiết nếu tìm thấy
    if columns_to_keep:
        df = df[columns_to_keep]
    else:
        print(f"No recognized columns to keep for file: {file_path}")
        return None
    
    # **Xử lý cột giá cả ('Price', 'Close', 'close')**
    for col in ['Price', 'Close', 'close']:
        if col in df.columns:
            # Loại bỏ các ký tự không cần thiết và chuyển đổi sang kiểu số
            df[col] = df[col].replace({'"': '', ',': ''}, regex=True)
            df[col] = df[col].replace({r'[^0-9.\-]': ''}, regex=True)
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
     # Xử lý cột thời gian ('Date', 'timeClose') và các cột thời gian khác nếu có
    if 'Date' in df.columns:
        try:
            df['Date'] = pd.to_datetime(df['Date'], utc=True, errors='coerce').dt.tz_convert(None).dt.date
        except Exception as e:
            print(f"Error converting 'Date' in {file_path}: {e}")
    
    if 'timeClose' in df.columns:
        try:
            df['timeClose'] = pd.to_datetime(df['timeClose'], utc=True, errors='coerce').dt.tz_convert(None).dt.date
        except Exception as e:
            print(f"Error converting 'timeClose' in {file_path}: {e}")
    
    return df



In [123]:
Bitcoin = load_prepare_data(r'1stJan2017 - 11thNov2024\crypto - format 1\Bitcoin_11_21_2016-1_20_2017_historical_data_coinmarketcap.csv')
SP500 = load_prepare_data(r'1stJan2017 - 11thNov2024/format 2/SP500.csv')
Gold = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\XAU_USD Historical Data.csv')
Silver = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\XAG_USD Historical Data (1).csv')
Tbond = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\United States 10-Year Bond Yield Historical Data (1).csv')
IMUS = load_prepare_data(r'1stJan2017 - 11thNov2024\format 2\IMUS.csv')
WTI = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\WTI_USD Historical Data (1).csv')
Dollar = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\US Dollar Index Historical Data (1).csv')
Franc = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\CHF_USD Historical Data (1).csv')
Ethereum = load_prepare_data(r'1stJan2017 - 11thNov2024\crypto - format 1\Ethereum_11_21_2016-1_20_2017_historical_data_coinmarketcap.csv')
Tether = load_prepare_data(r'1stJan2017 - 11thNov2024\crypto - format 1\Tether USDt_11_21_2016-1_20_2017_historical_data_coinmarketcap.csv')
GSCI=load_prepare_data(r'1stJan2017 - 11thNov2024\format 2\GSCI.csv')

Processing file: 1stJan2017 - 11thNov2024\crypto - format 1\Bitcoin_11_21_2016-1_20_2017_historical_data_coinmarketcap.csv
Columns: ['timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap', 'timestamp']
Processing file: 1stJan2017 - 11thNov2024/format 2/SP500.csv
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
Processing file: 1stJan2017 - 11thNov2024\format 3\XAU_USD Historical Data.csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: 1stJan2017 - 11thNov2024\format 3\XAG_USD Historical Data (1).csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: 1stJan2017 - 11thNov2024\format 3\United States 10-Year Bond Yield Historical Data (1).csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Change %']
Processing file: 1stJan2017 - 11thNov2024\format 2\IMUS.csv
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', '

In [124]:
Bitcoin = Bitcoin.rename(columns={'timeClose': 'Date'})
Ethereum = Ethereum.rename(columns={'timeClose': 'Date'})
Tether = Tether.rename(columns={'timeClose': 'Date'})

In [125]:
Bitcoin = Bitcoin.rename(columns={'close': 'Price'})
Ethereum = Ethereum.rename(columns={'close': 'Price'})
Tether = Tether.rename(columns={'close': 'Price'})


In [126]:
SP500=SP500.rename(columns={'Close': 'Price'})
IMUS=IMUS.rename(columns={'Close': 'Price'})
GSCI=GSCI.rename(columns={'Close': 'Price'})

# Segment data by phase

In [127]:
# Tạo dictionary cho tất cả các tài sản
assets = {
    "SP500": SP500,
    "Gold": Gold,
    "Silver": Silver,
    "Tbond": Tbond,
    "GSCI":GSCI,
    "IMUS": IMUS,
    "WTI": WTI,
    "Dollar": Dollar,
    "Franc": Franc,
    "Bitcoin": Bitcoin,
    "Ethereum": Ethereum,
    "Tether": Tether
}

Hàm tạo cột chỉ mục ( index) bằng ngày tháng thay vì số thứ tự . Điều này giúp cho việc kết hợp , phân chia dữ liệu được đồng về mặt thời gian

In [128]:
def convert_to_datetime_index_with_copy(assets, date_column='Date', copy_column_name='Time'):
    for name, df in assets.items():
        if date_column in df.columns:
            try:
                # Chuyển đổi cột 'Date' sang kiểu datetime
                df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
                
                # Loại bỏ các dòng có giá trị null trong cột 'Date' trước khi sao chép
                df.dropna(subset=[date_column], inplace=True)
                
                # Tạo một cột bản sao 'Time' từ cột 'Date'
                df[copy_column_name] = df[date_column]
                
                # Đặt cột 'Time' làm index
                df.set_index(copy_column_name, inplace=True)
                
                print(f"Đã chuyển '{date_column}' thành datetime và tạo index từ cột '{copy_column_name}' cho bộ dữ liệu {name}")
            except Exception as e:
                print(f"Lỗi khi xử lý bộ dữ liệu {name}: {e}")
        else:
            print(f"Bộ dữ liệu {name} không có cột '{date_column}'")
    return assets

# Sử dụng hàm với tập dữ liệu assets
assets = convert_to_datetime_index_with_copy(assets)

Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu SP500
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Gold
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Silver
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Tbond
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu GSCI
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu IMUS
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu WTI
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Dollar
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Franc
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Bitcoin
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Ethereum
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Tether


In [129]:
assets['Dollar']

,Date,Price
Time,,
2024-11-11,2024-11-11,105.54
2024-11-08,2024-11-08,105.00
2024-11-07,2024-11-07,104.51
2024-11-06,2024-11-06,105.09
2024-11-05,2024-11-05,103.42
...,...,...
2017-01-06,2017-01-06,102.22
2017-01-05,2017-01-05,101.52
2017-01-04,2017-01-04,102.70


hàm phân chia dữ liệu theo giai đoạn

In [130]:
def segment_data_by_period(assets, periods, date_column='Date'):
    # Khởi tạo từ điển để lưu dữ liệu đã phân đoạn
    segmented_data = {period_name: {} for period_name in periods}

    for period_name, (start_date, end_date) in periods.items():
        # Chuyển start_date và end_date sang datetime và normalize
        start_date = pd.to_datetime(start_date).normalize()
        end_date = pd.to_datetime(end_date).normalize()

        for name, df in assets.items():
            # Chuyển cột 'Date' thành datetime và normalize nếu cần
            if date_column in df.columns:
                df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()

            # Kiểm tra và in các giá trị min, max để xác nhận phạm vi thời gian
            print(f"Kiểm tra dữ liệu {name} trong giai đoạn {period_name}: {df[date_column].min()} - {df[date_column].max()}")

            # Tạo dải ngày đầy đủ cho giai đoạn hiện tại
            full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
            
            # Tạo DataFrame mới với dải ngày đầy đủ
            df_period = pd.DataFrame({date_column: full_date_range})
            
            # Hợp nhất dữ liệu gốc với dải ngày đầy đủ dựa trên cột 'Date'
            if date_column in df.columns:
                df_merged = pd.merge(df_period, df, on=date_column, how='left')
                segmented_data[period_name][name] = df_merged
                print(f"Đã hợp nhất dữ liệu cho {name} trong giai đoạn {period_name} với dải ngày đầy đủ")
            else:
                print(f"Dữ liệu {name} không có cột {date_column}, bỏ qua.")

    return segmented_data


In [131]:

covid_period = ('2020-01-05', '2023-05-07')
full_sample_period = ('2017-01-01', '2024-11-11')
before_crisic= ('2017-01-01', '2019-01-05')
after_crisic=('2023-05-07', '2024-11-11')
periods = {
    "Full sample": full_sample_period,
    "COVID-19": covid_period,
    "Before crisic": before_crisic,
    "After crisic":after_crisic
}

# Phân đoạn dữ liệu
segmented_data = segment_data_by_period(assets, periods)

Kiểm tra dữ liệu SP500 trong giai đoạn Full sample: 2017-01-03 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho SP500 trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu Gold trong giai đoạn Full sample: 2017-01-02 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho Gold trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu Silver trong giai đoạn Full sample: 2017-01-02 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho Silver trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu Tbond trong giai đoạn Full sample: 2017-01-02 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho Tbond trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu GSCI trong giai đoạn Full sample: 2017-01-03 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho GSCI trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu IMUS trong giai đoạn Full sample: 2017-01-03 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho IMUS trong giai đoạn Full

hàm kiểm tra dữ liệu sau khi phân chia

In [132]:
def check_segmented_data(segmented_data, periods, date_column='Date'):
    # Duyệt qua từng giai đoạn và tài sản đã phân đoạn
    for period_name, assets_in_period in segmented_data.items():
        print(f"Giai đoạn: {period_name}")
        start_date, end_date = periods[period_name]  # Lấy mốc thời gian của từng giai đoạn
        start_date = pd.to_datetime(start_date).normalize()
        end_date = pd.to_datetime(end_date).normalize()

        for asset_name, df in assets_in_period.items():
            if not df.empty:  # Kiểm tra nếu DataFrame không rỗng
                print(f"  Tài sản: {asset_name}")
                
                # Kiểm tra nếu cột 'Date' tồn tại
                if date_column in df.columns:
                    # Chuyển cột 'Date' thành datetime và chuẩn hóa
                    df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()
                    
                    # Lấy ngày bắt đầu và kết thúc từ cột 'Date'
                    min_date = df[date_column].min()
                    max_date = df[date_column].max()
                    
                    print(f"    Bắt đầu: {min_date}, Kết thúc: {max_date}")
                    
                    # Kiểm tra xem dữ liệu có nằm trong phạm vi giai đoạn không
                    if min_date >= start_date and max_date <= end_date:
                        print(f"    Dữ liệu hợp lệ trong phạm vi {start_date} đến {end_date}")
                    else:
                        print(f"    Lỗi: Dữ liệu không nằm trong phạm vi {start_date} đến {end_date}")
                else:
                    print(f"    Lỗi: Tài sản {asset_name} không có cột '{date_column}'")
            else:
                print(f"  Tài sản: {asset_name} - Không có dữ liệu trong giai đoạn này")
        print("-" * 40)


In [133]:
# Kiểm tra dữ liệu đã phân đoạn
check_segmented_data(segmented_data, periods)

Giai đoạn: Full sample
  Tài sản: SP500
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: Gold
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: Silver
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: Tbond
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: GSCI
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: IMUS
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: WTI
    Bắt đầu: 201

In [134]:
segmented_data['COVID-19']['GSCI']

,Date,Price
0,2020-01-05,NaN
1,2020-01-06,1.03730
2,2020-01-07,1.03500
3,2020-01-08,1.03180
4,2020-01-09,1.03240
...,...,...
1214,2023-05-03,1.13505
1215,2023-05-04,1.13470
1216,2023-05-05,1.12855
1217,2023-05-06,NaN


# Calculate daily profit for each period

Xử lý giá trị Price

In [135]:
def handle_missing_values(df, price_column='Price'):

    # Kiểm tra nếu cột 'Price' có trong DataFrame
    if price_column in df.columns:
        # Đầu tiên, xác định các khoảng thời gian liên tiếp bị thiếu
        df['Missing'] = df[price_column].isna()
        df['Missing_Group'] = (df['Missing'] != df['Missing'].shift()).cumsum()
        
        # Đếm số lượng ngày liên tiếp bị thiếu
        missing_counts = df.groupby('Missing_Group')['Missing'].transform('sum')
        
        # Sử dụng nội suy cho các khoảng thời gian bị thiếu dưới 3 ngày
        df.loc[(df['Missing']) & (missing_counts < 4), price_column] = df[price_column].interpolate(method='linear')
        
        # Xóa các cột tạm thời
        df.drop(['Missing', 'Missing_Group'], axis=1, inplace=True)
    
    return df

In [136]:

def log_returns(segmented_data, price_column='Price'):
    """
    Tính toán log return sau khi xử lý missing values cho từng giai đoạn và từng tài sản.
    """
    # Tạo dictionary để lưu trữ log returns cho từng giai đoạn
    log_returns_data = {period: {} for period in segmented_data}

    # Duyệt qua từng giai đoạn và từng tài sản
    for period, assets in segmented_data.items():
        for asset_name, df in assets.items():
            if price_column in df.columns:
                try:
                    # Xử lý missing values cho cột 'Price'
                    df = handle_missing_values(df, price_column)
                    
                    # Tính log return
                    df['Log Return'] = np.log(df[price_column] / df[price_column].shift(1))
                    
                    # Lưu DataFrame đã tính log return vào dictionary
                    log_returns_data[period][asset_name] = df
                    print(f"Tính Log Return cho {asset_name} trong giai đoạn {period} thành công.")
                except Exception as e:
                    print(f"Lỗi khi tính toán Log Return cho {asset_name} trong giai đoạn {period}: {e}")
            else:
                print(f"Tài sản {asset_name} không có cột {price_column}. Bỏ qua.")

    return log_returns_data

In [137]:
log_returns_data = log_returns(segmented_data)

# Kiểm tra kết quả
for period, assets in log_returns_data.items():
    for asset, df in assets.items():
        print(f"{asset} - {period}:")
        print(df[['Date', 'Price', 'Log Return']].head())

Tính Log Return cho SP500 trong giai đoạn Full sample thành công.
Tính Log Return cho Gold trong giai đoạn Full sample thành công.
Tính Log Return cho Silver trong giai đoạn Full sample thành công.
Tính Log Return cho Tbond trong giai đoạn Full sample thành công.
Tính Log Return cho GSCI trong giai đoạn Full sample thành công.
Tính Log Return cho IMUS trong giai đoạn Full sample thành công.
Tính Log Return cho WTI trong giai đoạn Full sample thành công.
Tính Log Return cho Dollar trong giai đoạn Full sample thành công.
Tính Log Return cho Franc trong giai đoạn Full sample thành công.
Tính Log Return cho Bitcoin trong giai đoạn Full sample thành công.
Tính Log Return cho Ethereum trong giai đoạn Full sample thành công.
Tính Log Return cho Tether trong giai đoạn Full sample thành công.
Tính Log Return cho SP500 trong giai đoạn COVID-19 thành công.
Tính Log Return cho Gold trong giai đoạn COVID-19 thành công.
Tính Log Return cho Silver trong giai đoạn COVID-19 thành công.
Tính Log Return 

In [138]:
log_returns_data['Full sample']['Dollar']

,Date,Price,Log Return
0,2017-01-01,NaN,NaN
1,2017-01-02,102.83,NaN
2,2017-01-03,103.21,0.003689
3,2017-01-04,102.70,-0.004954
4,2017-01-05,101.52,-0.011556
...,...,...,...
2867,2024-11-07,104.51,-0.005534
2868,2024-11-08,105.00,0.004678
2869,2024-11-09,105.18,0.001713
2870,2024-11-10,105.36,0.001710


Xử lý giá trị lợi nhuận 

In [139]:
# Hàm xử lý dữ liệu log return
def preprocess_log_returns(log_returns_data, column='Log Return'):
    """
    Xử lý log returns: loại bỏ giá trị 0, nội suy và loại bỏ NaN.
    """
    # Tạo dictionary để lưu trữ dữ liệu đã xử lý cho từng giai đoạn
    processed_data = {period: {} for period in log_returns_data}

    # Duyệt qua từng giai đoạn và từng tài sản trong log_returns_data
    for period, assets in log_returns_data.items():
        print(f"Đang xử lý log returns cho giai đoạn: {period}")
        for asset_name, asset_data in assets.items():
            # Kiểm tra sự tồn tại của cột 'Log Return'
            if column not in asset_data.columns:
                print(f"Tài sản {asset_name} không có cột '{column}'. Bỏ qua.")
                continue

            # Tạo bản sao của DataFrame để xử lý
            series = asset_data[['Date',column]].copy()
            
            # Thay giá trị 0 thành NaN
            series[column] = series[column].replace(0, np.nan)

            # Nội suy tuyến tính để điền các giá trị thiếu (NaN)
            series[column] = series[column].interpolate(method='linear')

            # Bỏ các giá trị NaN còn lại sau khi nội suy
            series = series.dropna()

            # Kiểm tra nếu dữ liệu sau khi xử lý còn đủ điểm dữ liệu
            if series.empty:
                print(f"Tài sản {asset_name} sau khi xử lý không có dữ liệu. Bỏ qua.")
                continue

            # Lưu lại DataFrame đã xử lý vào dictionary
            processed_data[period][asset_name] = series
            print(f"Đã xử lý log returns cho {asset_name} trong giai đoạn {period} thành công.")

    return processed_data




In [140]:
Log_return_cleared=preprocess_log_returns(log_returns_data)

Đang xử lý log returns cho giai đoạn: Full sample
Đã xử lý log returns cho SP500 trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Gold trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Silver trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Tbond trong giai đoạn Full sample thành công.
Đã xử lý log returns cho GSCI trong giai đoạn Full sample thành công.
Đã xử lý log returns cho IMUS trong giai đoạn Full sample thành công.
Đã xử lý log returns cho WTI trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Dollar trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Franc trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Bitcoin trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Ethereum trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Tether trong giai đoạn Full sample thành công.
Đang xử lý log returns cho giai đoạn: COVID-19
Đã xử lý log returns cho SP500 trong giai đoạn C

In [163]:
Log_return_cleared['COVID-19']['Bitcoin']

,Date,Log Return
1,2020-01-06,0.047161
2,2020-01-07,0.049527
3,2020-01-08,-0.010322
4,2020-01-09,-0.025165
5,2020-01-10,0.035837
...,...,...
1214,2023-05-03,0.011295
1215,2023-05-04,-0.005483
1216,2023-05-05,0.023524
1217,2023-05-06,-0.021554


In [162]:
Log_return_cleared['COVID-19']['Bitcoin']

,Date,Log Return
1,2020-01-06,0.047161
2,2020-01-07,0.049527
3,2020-01-08,-0.010322
4,2020-01-09,-0.025165
5,2020-01-10,0.035837
...,...,...
1214,2023-05-03,0.011295
1215,2023-05-04,-0.005483
1216,2023-05-05,0.023524
1217,2023-05-06,-0.021554


*  dữ liệu cuối  trước khi đưua vào model : Log_return_cleared

#  Descriptive statistics

In [142]:
from scipy.stats import skew, kurtosis, jarque_bera

In [143]:
def generate_summary_table(Log_return_cleared):
    """
    Hàm này tạo bảng thống kê bao gồm các thống kê tổng hợp và kiểm định Jarque-Bera.
    Các ký hiệu *, **, và *** tương ứng với mức ý nghĩa 1%, 5%, và 10%.
    """
    data_list = []

    # Duyệt qua từng giai đoạn và từng tài sản trong Log_return_cleared
    for period, data_dict in Log_return_cleared.items():
        print(f"Đang tạo bảng tóm tắt cho giai đoạn: {period}")
        for asset, df in data_dict.items():
            # Kiểm tra nếu cột 'Log Return' tồn tại và loại bỏ các giá trị NaN
            if 'Log Return' in df.columns:
                df_cleaned = df['Log Return'].dropna()

                # Kiểm tra nếu còn đủ dữ liệu để tính toán
                if len(df_cleaned) > 0:
                    # Tính các thống kê cho từng tài sản và từng giai đoạn
                    mean = df_cleaned.mean()
                    std_dev = df_cleaned.std()
                    sharpe_ratio = mean / std_dev if std_dev != 0 else 0
                    skewness = df_cleaned.skew()
                    kurtosis = df_cleaned.kurtosis()

                    # Kiểm định Jarque-Bera
                    jb_stat, jb_p_value = jarque_bera(df_cleaned)

                    # Đánh dấu mức ý nghĩa thống kê
                    if jb_p_value < 0.01:
                        jb_significance = '*'
                    elif jb_p_value < 0.05:
                        jb_significance = '**'
                    elif jb_p_value < 0.10:
                        jb_significance = '***'
                    else:
                        jb_significance = ''

                    # Lưu kết quả vào danh sách
                    data_list.append({
                        'Variable': asset,
                        'Period': period,
                        'Mean': mean,
                        'Std. Dev.': std_dev,
                        'Sharpe Ratio': sharpe_ratio,
                        'Skewness': skewness,
                        'Kurtosis': kurtosis,
                        'Jarque-Bera': f"{jb_stat:.4f}{jb_significance}"
                    })

    # Chuyển kết quả thành DataFrame
    summary_df = pd.DataFrame(data_list)

    return summary_df

# Sử dụng hàm generate_summary_table với dữ liệu đã được xử lý
summary_df = generate_summary_table(Log_return_cleared)

Đang tạo bảng tóm tắt cho giai đoạn: Full sample
Đang tạo bảng tóm tắt cho giai đoạn: COVID-19
Đang tạo bảng tóm tắt cho giai đoạn: Before crisic
Đang tạo bảng tóm tắt cho giai đoạn: After crisic


In [144]:
summary_df

,Variable,Period,Mean,Std. Dev.,Sharpe Ratio,Skewness,Kurtosis,Jarque-Bera
0,SP500,Full sample,3.410247e-04,0.008929,0.038191,-0.266825,18.616918,41312.4073*
1,Gold,Full sample,2.867723e-04,0.006736,0.042574,-0.346769,6.343315,4849.3038*
2,Silver,Full sample,2.709249e-04,0.013466,0.020119,-0.569044,12.400918,18474.4072*
3,Tbond,Full sample,2.754639e-04,0.024926,0.011051,1.160760,43.753505,228752.0638*
4,GSCI,Full sample,6.889009e-05,0.003694,0.018650,0.682693,6.746386,5641.1086*
5,IMUS,Full sample,6.889009e-05,0.003694,0.018650,0.682693,6.746386,5641.1086*
6,WTI,Full sample,6.747355e-05,0.023666,0.002851,-1.424732,79.885619,761154.8043*
7,Dollar,Full sample,4.706796e-05,0.003305,0.014241,-0.127268,4.028297,1939.4779*
8,Franc,Full sample,4.226003e-05,0.003607,0.011718,0.415202,4.897529,2938.2522*
9,Bitcoin,Full sample,1.571214e-03,0.037891,0.041467,-0.668822,11.298706,15426.4811*


#  unit-root tests

In [145]:
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import PhillipsPerron

def run_unit_root_tests(Log_return_cleared):
    data_list = []

    # Duyệt qua từng giai đoạn và từng tài sản
    for period, data_dict in Log_return_cleared.items():
        for asset, df in data_dict.items():
            if 'Log Return' in df.columns:
                
                # Kiểm tra nếu còn đủ dữ liệu để tính toán và không chứa NaN/Inf
                log_returns = df['Log Return'].dropna()
                
                if len(log_returns) > 0:
                    # Kiểm định ADF
                    adf_stat, adf_p_value, _, _, _, _ = adfuller(log_returns)
                    
                    # Đánh dấu mức ý nghĩa ADF
                    if adf_p_value < 0.01:
                        adf_significance = '*'
                    elif adf_p_value < 0.05:
                        adf_significance = '**'
                    elif adf_p_value < 0.10:
                        adf_significance = '***'
                    else:
                        adf_significance = ''
                    
                    # Kiểm định PP
                    pp_result = PhillipsPerron(log_returns)
                    pp_stat, pp_p_value = pp_result.stat, pp_result.pvalue
                    
                    # Đánh dấu mức ý nghĩa PP
                    if pp_p_value < 0.01:
                        pp_significance = '*'
                    elif pp_p_value < 0.05:
                        pp_significance = '**'
                    elif pp_p_value < 0.10:
                        pp_significance = '***'
                    else:
                        pp_significance = ''
                    
                    # Lưu kết quả vào list
                    data_list.append({
                        'Variable': asset,
                        'Period': period,
                        'ADF': f"{adf_stat:.4f}{adf_significance}",
                        'ADF p-value': adf_p_value,
                        'PP': f"{pp_stat:.4f}{pp_significance}",
                        'PP p-value': pp_p_value
                    })

    # Chuyển kết quả thành DataFrame
    summary_df = pd.DataFrame(data_list)


    
    return summary_df

# Ví dụ sử dụng hàm
summary_unit_root_df = run_unit_root_tests(log_returns_data)

In [146]:
summary_unit_root_df

,Variable,Period,ADF,ADF p-value,PP,PP p-value
0,SP500,Full sample,-11.9735*,3.848720e-22,-55.7762*,0.0
1,Gold,Full sample,-51.9704*,0.000000e+00,-51.9486*,0.0
2,Silver,Full sample,-12.2977*,7.571353e-23,-51.1328*,0.0
3,Tbond,Full sample,-8.1491*,9.803429e-13,-51.7694*,0.0
4,GSCI,Full sample,-48.8937*,0.000000e+00,-48.6979*,0.0
5,IMUS,Full sample,-48.8937*,0.000000e+00,-48.6979*,0.0
6,WTI,Full sample,-10.2273*,5.137502e-18,-46.3869*,0.0
7,Dollar,Full sample,-50.9190*,0.000000e+00,-50.8544*,0.0
8,Franc,Full sample,-49.3887*,0.000000e+00,-49.2271*,0.0
9,Bitcoin,Full sample,-36.8925*,0.000000e+00,-55.1842*,0.0


# Merge Data

In [147]:
def merge_assets_with_sp500(Log_return_cleared, assets_to_merge=['Bitcoin', 'Gold', 'Silver', 'Tbond', 
                                                                'Dollar', 'IMUS', 'WTI', 'Franc', 
                                                                'Ethereum', 'Tether']):
    """
    Hàm này hợp nhất log return của các tài sản với S&P 500 cho từng giai đoạn.
    """
    # Tạo dictionary để lưu trữ kết quả merge cho từng giai đoạn
    merged_data_dict = {period: {} for period in Log_return_cleared}
    
    for period, assets_data in Log_return_cleared.items():
        # Kiểm tra nếu S&P 500 có trong dữ liệu
        if 'SP500' not in assets_data:
            print(f"Không tìm thấy dữ liệu S&P 500 cho giai đoạn {period}. Bỏ qua.")
            continue

        # Lấy log return của S&P 500 và đổi tên cột
        sp500_log_return = assets_data['SP500'].rename(columns={'Log Return': 'SP500_Log_Return'})

        # Hợp nhất từng tài sản với S&P 500
        for asset in assets_to_merge:
            if asset in assets_data:
                try:
                    # Lấy dữ liệu của tài sản
                    asset_data = assets_data[asset]
                    
                    # Kiểm tra nếu cột 'Log Return' tồn tại
                    if 'Log Return' not in asset_data.columns:
                        print(f"Tài sản {asset} không có cột 'Log Return' trong giai đoạn {period}. Bỏ qua.")
                        continue

                    # Merge dữ liệu dựa trên cột 'Date'
                    merged_data = pd.merge(asset_data, sp500_log_return, on='Date', how='left')
                    merged_data_dict[period][asset.lower()] = merged_data

                    print(f"Merge thành công cho {asset} trong giai đoạn {period}.")
                except Exception as e:
                    print(f"Lỗi khi merge {asset} trong giai đoạn {period}: {e}")
            else:
                print(f"Tài sản {asset} không tồn tại trong giai đoạn {period}.")
    
    return merged_data_dict

# Gọi hàm để hợp nhất dữ liệu
merged_data_dict = merge_assets_with_sp500(Log_return_cleared)

# Hiển thị kết quả cho từng tài sản (nếu cần)
for period, assets in merged_data_dict.items():
    for asset, data in assets.items():
        print(f"Merged data for {asset} in {period}:\n", data[['Date', 'Log Return', 'SP500_Log_Return']].head())


Merge thành công cho Bitcoin trong giai đoạn Full sample.
Merge thành công cho Gold trong giai đoạn Full sample.
Merge thành công cho Silver trong giai đoạn Full sample.
Merge thành công cho Tbond trong giai đoạn Full sample.
Merge thành công cho Dollar trong giai đoạn Full sample.
Merge thành công cho IMUS trong giai đoạn Full sample.
Merge thành công cho WTI trong giai đoạn Full sample.
Merge thành công cho Franc trong giai đoạn Full sample.
Merge thành công cho Ethereum trong giai đoạn Full sample.
Merge thành công cho Tether trong giai đoạn Full sample.
Merge thành công cho Bitcoin trong giai đoạn COVID-19.
Merge thành công cho Gold trong giai đoạn COVID-19.
Merge thành công cho Silver trong giai đoạn COVID-19.
Merge thành công cho Tbond trong giai đoạn COVID-19.
Merge thành công cho Dollar trong giai đoạn COVID-19.
Merge thành công cho IMUS trong giai đoạn COVID-19.
Merge thành công cho WTI trong giai đoạn COVID-19.
Merge thành công cho Franc trong giai đoạn COVID-19.
Merge thành 

In [148]:
merge_result=preprocess_log_returns(merged_data_dict, column='Log Return')

Đang xử lý log returns cho giai đoạn: Full sample
Đã xử lý log returns cho bitcoin trong giai đoạn Full sample thành công.
Đã xử lý log returns cho gold trong giai đoạn Full sample thành công.
Đã xử lý log returns cho silver trong giai đoạn Full sample thành công.
Đã xử lý log returns cho tbond trong giai đoạn Full sample thành công.
Đã xử lý log returns cho dollar trong giai đoạn Full sample thành công.
Đã xử lý log returns cho imus trong giai đoạn Full sample thành công.
Đã xử lý log returns cho wti trong giai đoạn Full sample thành công.
Đã xử lý log returns cho franc trong giai đoạn Full sample thành công.
Đã xử lý log returns cho ethereum trong giai đoạn Full sample thành công.
Đã xử lý log returns cho tether trong giai đoạn Full sample thành công.
Đang xử lý log returns cho giai đoạn: COVID-19
Đã xử lý log returns cho bitcoin trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho gold trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho silver trong giai đoạn COVI

# Regressionn

# ENHANCED LSTM WITH COVID 19 DATA

In [ ]:
# prepare covid data

In [ ]:

def load_prepare_data_covid19_weekly_sundays_only(file_path, start_date='2020-01-05', end_date='2023-05-07'):
    # Load CSV file
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None
    
    # Convert 'date' column to datetime format for easy manipulation
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    
    # Filter data to the COVID timespan
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    # Drop rows with missing essential values
    df.dropna(subset=['date', 'continent', 'total_cases', 'total_deaths', 'total_cases_per_million', 'total_deaths_per_million'], inplace=True)
    
    # Filter out countries with incomplete data in 'total_cases' or 'total_deaths'
    complete_countries = df.groupby('location').filter(lambda x: x['total_cases'].notna().all() and x['total_deaths'].notna().all())['location'].unique()
    df = df[df['location'].isin(complete_countries)]
    
    # Group by 'date' and 'continent', summing the relevant columns
    df_continent_daily = df.groupby(['date', 'continent']).agg({
        'total_cases': 'sum',
        'total_deaths': 'sum',
        'total_cases_per_million': 'sum',
        'total_deaths_per_million': 'sum'
    }).reset_index()
    
    # Filter for Sundays only
    df_sundays = df_continent_daily[df_continent_daily['date'].dt.dayofweek == 6].copy()
    
    # Calculate the weekly change
    df_sundays['delta_total_cases_per_million_weekly'] = df_sundays.groupby('continent')['total_cases_per_million'].diff().fillna(0)
    
    # Aggregate across all continents for each Sunday
    df_weekly_summary = df_sundays.groupby('date').agg({
        'total_cases': 'sum',
        'total_deaths': 'sum',
        'total_cases_per_million': 'sum',
        'total_deaths_per_million': 'sum',
        'delta_total_cases_per_million_weekly': 'sum'
    }).reset_index()

    # Upsample to daily data by forward filling
    df_weekly_summary.set_index('date', inplace=True)
    df_daily = df_weekly_summary.resample('D').ffill().reset_index()
    
    # Rename columns for clarity
    df_daily.rename(columns={
        'date': 'Date',
        'total_cases': 'Total_Cases',
        'total_deaths': 'Total_Deaths',
        'total_cases_per_million': 'Total_Cases_per_Million',
        'total_deaths_per_million': 'Total_Deaths_per_Million',
        'delta_total_cases_per_million_weekly': 'Delta_Total_Cases_per_Million_Weekly'
    }, inplace=True)
    
    # Select and reorder columns
    df_daily = df_daily[['Date', 'Total_Cases', 'Total_Deaths', 'Total_Cases_per_Million', 'Total_Deaths_per_Million', 'Delta_Total_Cases_per_Million_Weekly']]
    
    return df_daily

# Usage example
file_path = r'Crisis\owid-covid-data.csv'  # Replace with actual path
covid_data_daily_summary = load_prepare_data_covid19_weekly_sundays_only(file_path)
print(covid_data_daily_summary)


           Date  Total_Cases  Total_Deaths  Total_Cases_per_Million  \
0    2020-01-05          2.0           3.0                     0.01   
1    2020-01-06          2.0           3.0                     0.01   
2    2020-01-07          2.0           3.0                     0.01   
3    2020-01-08          2.0           3.0                     0.01   
4    2020-01-09          2.0           3.0                     0.01   
...         ...          ...           ...                      ...   
1214 2023-05-03  764409963.0     6927839.0              46829358.44   
1215 2023-05-04  764409963.0     6927839.0              46829358.44   
1216 2023-05-05  764409963.0     6927839.0              46829358.44   
1217 2023-05-06  764409963.0     6927839.0              46829358.44   
1218 2023-05-07  765019412.0     6932563.0              46873804.00   

      Total_Deaths_per_Million  Delta_Total_Cases_per_Million_Weekly  
0                         0.04                                  0.00  
1    

# method 1 : transformer model 

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, TimeDistributed
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# List of assets to include in training
assets = ["Bitcoin", "Gold", "Silver", "T_Bonds"]  # Replace with actual asset variable names

# Load COVID-19 data (interpolated to daily frequency)
file_path = r'Crisis\owid-covid-data.csv'  # Replace with actual path
covid_data_daily_summary = load_prepare_data_covid19_weekly_sundays_only(file_path)

# Combine all asset data with COVID-19 data
combined_data_list = []
for asset_name in assets:
    asset_data = globals()[asset_name]  # Access each asset DataFrame by its name
    asset_data['Date'] = pd.to_datetime(asset_data['Date'])
    asset_data['Log_Return'] = np.log(asset_data['Price'] / asset_data['Price'].shift(1))
    merged_data = pd.merge(asset_data, covid_data_daily_summary, on='Date', how='inner')
    combined_data_list.append(merged_data[['Log_Return', 'Total_Cases', 'Total_Deaths', 
                                           'Total_Cases_per_Million', 'Total_Deaths_per_Million', 
                                           'Delta_Total_Cases_per_Million_Weekly']])

# Concatenate data for all assets
combined_data = pd.concat(combined_data_list, ignore_index=True)

# Scale data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)
X = scaled_data[:-1]
y = scaled_data[1:, 0]  # Target is next day's log return

# Reshape for Transformer model
X = X.reshape((1, X.shape[0], X.shape[1]))
y = y.reshape((1, y.shape[0], 1))

# Define Transformer model
def build_transformer_model(input_shape, num_heads=2, ff_dim=32):
    inputs = Input(shape=input_shape)
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(num_heads=num_heads, key_dim=input_shape[-1])(x, x)
    x = Dropout(0.1)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = TimeDistributed(Dense(ff_dim, activation='relu'))(x)
    outputs = TimeDistributed(Dense(1))(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

# Train model
input_shape = (X.shape[1], X.shape[2])
transformer_model = build_transformer_model(input_shape)
transformer_model.fit(X, y, epochs=10, batch_size=1, validation_split=0.2)

# Predict and plot results
predicted_log_returns = transformer_model.predict(X)
predicted_log_returns = scaler.inverse_transform(
    np.concatenate([predicted_log_returns[0], np.zeros((predicted_log_returns.shape[1], X.shape[2] - 1))], axis=1))[:, 0]

plt.figure(figsize=(12, 6))
plt.plot(range(len(y[0])), scaler.inverse_transform(y[0]), label="Actual Log Returns")
plt.plot(range(len(predicted_log_returns)), predicted_log_returns, label="Predicted Log Returns", linestyle="dashed")
plt.title("Transformer Model Prediction on Log Returns for Multiple Assets")
plt.xlabel("Time")
plt.ylabel("Log Return")
plt.legend()
plt.show()


KeyError: 'T_Bonds'

# method 2 : Sequence to sequence LSTM

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Attention, Input
import tensorflow as tf

# Prepare data for multiple assets
sequence_length = len(covid_data_daily_summary)  # Full sequence length for crisis period
combined_data_list = []
for asset_name in assets:
    asset_data = globals()[asset_name]  # Access each asset DataFrame by name
    asset_data['Date'] = pd.to_datetime(asset_data['Date'])
    asset_data['Log_Return'] = np.log(asset_data['Price'] / asset_data['Price'].shift(1))
    merged_data = pd.merge(asset_data, covid_data_daily_summary, on='Date', how='inner')
    combined_data_list.append(merged_data[['Log_Return', 'Total_Cases', 'Total_Deaths', 
                                           'Total_Cases_per_Million', 'Total_Deaths_per_Million', 
                                           'Delta_Total_Cases_per_Million_Weekly']])

# Concatenate data for all assets
combined_data = pd.concat(combined_data_list, ignore_index=True)

# Scale data and create sequences for Seq2Seq model
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)
X, y = [], []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(scaled_data[i, 0])  # Target is the next log return
X, y = np.array(X), np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Define Seq2Seq LSTM with Attention
def build_seq2seq_lstm_with_attention(input_shape):
    encoder_inputs = Input(shape=input_shape)
    encoder_lstm = LSTM(64, return_sequences=True)(encoder_inputs)
    encoder_output, state_h, state_c = LSTM(64, return_state=True)(encoder_lstm)
    
    decoder_inputs = RepeatVector(input_shape[0])(encoder_output)
    decoder_lstm = LSTM(64, return_sequences=True)(decoder_inputs)
    attention = Attention()([decoder_lstm, encoder_output])
    attention_output = tf.keras.layers.Concatenate(axis=-1)([attention, decoder_lstm])
    outputs = TimeDistributed(Dense(1))(attention_output)
    
    model = Model(encoder_inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

# Train Seq2Seq LSTM model with Attention
input_shape = (X_train.shape[1], X_train.shape[2])
seq2seq_lstm_attention_model = build_seq2seq_lstm_with_attention(input_shape)
seq2seq_lstm_attention_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Predict and plot results
predicted_log_returns = seq2seq_lstm_attention_model.predict(X_test)
predicted_log_returns = scaler.inverse_transform(
    np.concatenate([predicted_log_returns.reshape(predicted_log_returns.shape[0], -1, 1), np.zeros((predicted_log_returns.shape[0], X.shape[2] - 1))], axis=2))[:, :, 0]

plt.figure(figsize=(12, 6))
plt.plot(range(len(y_test)), scaler.inverse_transform(y_test.reshape(-1, 1)), label="Actual Log Returns")
plt.plot(range(len(predicted_log_returns)), predicted_log_returns[:, 0], label="Predicted Log Returns", linestyle="dashed")
plt.title("Seq2Seq LSTM with Attention Prediction on Log Returns for Multiple Assets")
plt.xlabel("Time")
plt.ylabel("Log Return")
plt.legend()
plt.show()
